O gp_minimize faz parte da biblioteca scikit-optimize (skopt) e implementa Bayesian Optimization usando Gaussian Processes. Ele é usado para otimização de hiperparâmetros em Machine Learning, tentando encontrar automaticamente os melhores valores em um espaço de busca.

In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 4.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from skopt.utils import use_named_args

# Dataset de exemplo
X, y = load_breast_cancer(return_X_y=True)

# Espaço de busca dos hiperparâmetros
space  = [
    Integer(10, 200, name="n_estimators"),        # número de árvores
    Integer(1, 20, name="max_depth"),             # profundidade máxima
    Real(0.01, 1.0, name="max_features"),         # proporção de features usadas
]

# Modelo base
rf = RandomForestClassifier(random_state=42)

# Função objetivo (negativa porque gp_minimize minimiza por padrão)
@use_named_args(space)
def objective(**params):
    rf.set_params(**params)
    scores = cross_val_score(rf, X, y, cv=5, scoring="accuracy")
    return -np.mean(scores)   # queremos maximizar acurácia → minimizar -acurácia

# Rodando a otimização
res = gp_minimize(objective, space, n_calls=30, random_state=42)

# Resultados
print("Melhor acurácia:", -res.fun)
print("Melhores hiperparâmetros:", res.x)
print("Nomeados:", dict(zip([dim.name for dim in space], res.x)))

Melhor acurácia: 0.9648657040832169
Melhores hiperparâmetros: [np.int64(97), np.int64(7), 0.1514381497427214]
Nomeados: {'n_estimators': np.int64(97), 'max_depth': np.int64(7), 'max_features': 0.1514381497427214}
